In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
import pandas as pd
from account.models import User, InstructorGroup

In [2]:
User.objects.get(id=501)

<User: 501 - last_name first_name>

In [ ]:
student_id = 5
instructor_id = 801
for i in range(1, 81):
    student_list = []
    if i <=40:
        for a in range(4):
            s = User.objects.get(id=student_id)
            student_list.append(s)
            student_id+=1
    else:
        for a in range(3):
            s = User.objects.get(id=student_id)
            student_list.append(s)
            student_id+=1
    instructor = User.objects.get(id=instructor_id)
    instructor_id += 1
    new_group = InstructorGroup.objects.create(id=i)
    new_group.instructor.set([instructor])
    new_group.students.set(student_list)
    new_group.save()

In [3]:
# Instructor google sheet
student_url = 'https://docs.google.com/spreadsheets/d/' + \
                   '1whM8CflvlkdVNdfdaaAAuzVn4YAMmCrws48Djrt8ONw' + \
                   '/export?gid=1768285988&format=csv'
student_df = pd.read_csv(student_url)
print(student_df.head())

instructor_url = 'https://docs.google.com/spreadsheets/d/' + \
                   '1whM8CflvlkdVNdfdaaAAuzVn4YAMmCrws48Djrt8ONw' + \
                   '/export?gid=1565332943&format=csv'
instructor_df = pd.read_csv(instructor_url)
print(instructor_df.head())

   id  組別 級別  總人數       school  學校人數 學生姓名 last_name first_name  班級 男/女  room
0   1   1  A   31   禮賢會彭學高紀念中學     1  歐韋君         歐         韋君  5A   女  C201
1   2   1  A   61   禮賢會彭學高紀念中學    31  馬苡晴         馬         苡晴  4C   女  C201
2   3   1  A  206  香港布廠商會朱石麟中學     2  陳楚澄         陳         楚澄  F3   女  B204
3   4   1  A  257       深圳培僑書院     6  肖瀑峰         肖         瀑峰  F2   男  D301
4   5   2  A   32   禮賢會彭學高紀念中學     2  劉鍵澄         劉         鍵澄  5A   女  C201
    id   屬會名稱： 姓名(中文)     姓名 (英文) last_name first_name 手提電話 (Whats app適用)  \
0  801  中華傳統文化    謝燕珊     Ada Tze         謝         燕珊           61312254   
1  802     NaN    陳曉君  Adele Chan         陳         曉君           62739203   
2  803      米埔    譚鎭誠    Alan Tam         譚         鎭誠           90882629   
3  804     九龍塘    吳梓騫    Alice Ng         吳         梓騫           64998861   
4  805     西九龍    黎雅麗    Ally Lai         黎         雅麗           91271952   

  電郵 Email (用於PORTAL導師登入)  instructor_group                      encrypted 

In [38]:
for i, r in instructor_df.iterrows():
    i_id = r['id']
    user = User.objects.get(id=i_id)
    user.first_name = r['first_name']
    user.last_name = r['last_name']
    user.save()
    
    group = r['instructor_group']
    instructor_group = InstructorGroup.objects.get(id=group)
    instructor_group.instructor.set(list(instructor_group.instructor.all()) + [user])
    instructor_group.save()

['a', 'b']

In [49]:
# instructor_db_df = pd.DataFrame(User.objects.filter(id__gte=801).filter(id__lte=873).all().values())
# instructor_db_df['url'] = "https://lionslifegame.app/player/" + instructor_db_df['encrypted_id']
# instructor_db_df['name'] = instructor_db_df['last_name'] + instructor_db_df['first_name']
# instructor_print_df = instructor_db_df[['id', 'name', 'url']]
# for i, r in instructor_print_df.iterrows():
#     gp = InstructorGroup.objects.filter(instructor=User.objects.get(id=r['id'])).first()
#     instructor_print_df.loc[i, 'instructor_group'] = int(gp.id)
instructor_print_df.to_excel('instructor.xlsx', index=None)

In [39]:
for i, r in student_df.iterrows():
    i_id = r['id']
    user = User.objects.get(id=i_id)
    user.first_name = r['first_name']
    user.last_name = r['last_name']
    user.school = r['school']
    user.room_no = r['room']
    user.save()
    
    group = r['組別']
    instructor_group = InstructorGroup.objects.get(id=group)
    instructor_group.students.set(list(instructor_group.students.all()) + [user])
    instructor_group.save()

In [55]:
student_group_df = student_df[['id', '組別']]

,id,first_name,last_name,url,room_no,組別
0,1,韋君,歐,https://lionslifegame.app/player/MNGVf0LX85DGo...,C201,1
1,2,苡晴,馬,https://lionslifegame.app/player/DytEn5MeccgNZ...,C201,1
2,3,楚澄,陳,https://lionslifegame.app/player/UyXFQPezBWt6L...,B204,1
3,4,瀑峰,肖,https://lionslifegame.app/player/ZWfo4udwt9OKM...,D301,1
4,5,鍵澄,劉,https://lionslifegame.app/player/IO4ktbw59lLww...,C201,2
...,...,...,...,...,...,...
275,276,銘明,謝,https://lionslifegame.app/player/7rpS8CSzrlMdU...,E202,69
276,277,婧雯,陳,https://lionslifegame.app/player/a70ohuWfT1qi5...,C303,67
277,278,穎晴,羅,https://lionslifegame.app/player/d3OUtVpjyNvyR...,B102,68
278,279,栩柔,李,https://lionslifegame.app/player/weKzI7goQqX4o...,C106,65


In [59]:
student_db_df = pd.DataFrame(User.objects.filter(id__gte=1).filter(id__lte=280).all().values())
student_db_df['url'] = "https://lionslifegame.app/player/" + student_db_df['encrypted_id']
student_db_df['name'] = student_db_df['last_name'] + student_db_df['first_name']
student_print_df = student_db_df[['id', 'name', 'url', 'room_no']]
student_print_df = student_print_df.merge(student_group_df, on='id')
student_print_df.to_excel('student.xlsx', index=None)

In [61]:
for i, r in student_df.iterrows():
    i_id = r['id']
    user = User.objects.get(id=i_id)
    user.best_booth = None
    user.save()

In [3]:
# VIP

In [5]:

vip_url = 'https://docs.google.com/spreadsheets/d/' + \
                   '1whM8CflvlkdVNdfdaaAAuzVn4YAMmCrws48Djrt8ONw' + \
                   '/export?gid=1465269158&format=csv'
vip_df = pd.read_csv(vip_url)
print(vip_df.head())

    id     name last_name  first_name
0  501   一心集團 1      一心集團           1
1  502   一心集團 2      一心集團           2
2  503   一心集團 3      一心集團           3
3  504   一心集團 4      一心集團           4
4  505  中區獅子會 1     中區獅子會           1


In [6]:
for i, r in vip_df.iterrows():
    i_id = r['id']
    user = User.objects.get(id=i_id)
    user.first_name = r['first_name']
    user.last_name = r['last_name']
    user.save()

In [7]:
vip_db_df = pd.DataFrame(User.objects.filter(id__gte=501).filter(id__lte=592).all().values())
vip_db_df['url'] = "https://lionslifegame.app/player/" + vip_db_df['encrypted_id']
vip_db_df['name'] = vip_db_df['last_name'] + vip_db_df['first_name']
vip_print_df = vip_db_df[['id', 'name', 'url']]
vip_print_df.to_excel('vip.xlsx', index=None)